## Hindsight Experience Replay (HER)

In [ ]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    xvfb \
    libosmesa6-dev \
    software-properties-common \
    patchelf

!pip install \
    free-mujoco-py \
    gym==0.21 \
    pytorch-lightning==1.6.0 \
    pyvirtualdisplay \
    PyOpenGL \
    PyOpenGL-accelerate

#### Setup virtual display

In [1]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

#### Import the necessary code libraries

In [2]:
import copy
import gymnasium as gym
import gymnasium_robotics
import torch
import itertools
import random

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW
from torch.distributions.normal import Normal

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics


device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()

In [3]:
device

'cuda:0'

In [4]:
def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

#### Create the replay buffer

In [5]:
class ReplayBuffer:

  def __init__(self, capacity, her_probability=0.8):#her_probability will decided
    #how often we will select samples of experience crated by hindsight experience replay
    self.her_probability = her_probability
    self.buffer = deque(maxlen=capacity//2)
    self.her_buffer = deque(maxlen=capacity//2)

  def __len__(self):
    return len(self.buffer) + len(self.her_buffer)

  def append(self, experience, her=False):
    if her:
      self.her_buffer.append(experience)
    else:
      self.buffer.append(experience)

  def sample(self, batch_size):
    her_batch_size = int(batch_size * self.her_probability)#we are calculating how many
    #number of entries will be in her_buffer and how many in normal buffer
    regular_batch_size = batch_size - her_batch_size

    batch = random.sample(self.buffer, regular_batch_size)#this batch samples from
    #normal buffer
    her_batch = random.sample(self.her_buffer, her_batch_size)
    #this batch samples from her_buffer
    full_batch = list(batch + her_batch)
    #at the end, we combine the two batches
    random.shuffle(full_batch)#just to ensure that every batch of this buffer that our
    #neural network takes as input contains both kind of entires
    return full_batch

In [6]:
class RLDataset(IterableDataset):

  def __init__(self, buffer, sample_size=400):
    self.buffer = buffer
    self.sample_size = sample_size
  
  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield experience

#### Create the environment

In [7]:
class RepeatActionWrapper(gym.Wrapper):
  def __init__(self, env, n):
    super().__init__(env)
    self.env = env
    self.n = n
      
  def step(self, action):
    done = False
    total_reward = 0.0
    for _ in range(self.n):
      next_state, reward, done, info = self.env.step(action)
      total_reward += reward
      if done:
        break
    return next_state, total_reward, done, info

In [9]:
def create_environment(name):
  env = gym.make(name, render_mode='rgb_array')
  env = RecordVideo(env, video_folder='./videos4', episode_trigger=lambda x: x % 50 == 0)
  # env = RepeatActionWrapper(env, n=4)
  env = RecordEpisodeStatistics(env)
  return env

#### Update the target network

In [10]:
def polyak_average(net, target_net, tau=0.01):
    for qp, tp in zip(net.parameters(), target_net.parameters()):
        tp.data.copy_(tau * qp.data + (1 - tau) * tp.data)

#### Create the Deep Q-Network

In [11]:
class DQN(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(obs_size + out_dims, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU(),           
        nn.Linear(hidden_size, 1),
    )

  def forward(self, state, action):
    if isinstance(state, np.ndarray):
      state = torch.from_numpy(state).to(device)
    if isinstance(action, np.ndarray):
      action = torch.from_numpy(action).to(device)
    in_vector = torch.hstack((state, action))
    return self.net(in_vector.float())

#### Create the gradient policy

In [13]:
class GradientPolicy(nn.Module):

  def __init__(self, hidden_size, obs_size, out_dims, max):
    super().__init__()

    self.max = torch.from_numpy(max).to(device)

    self.net = nn.Sequential(
        nn.Linear(obs_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU()
    )
    self.linear_mu = nn.Linear(hidden_size, out_dims)
    # self.linear_std = nn.Linear(hidden_size, out_dims)
    self.linear_log_std = nn.Linear(hidden_size, out_dims)

  def forward(self, obs):
    if isinstance(obs, np.ndarray):
      obs = torch.from_numpy(obs).to(device)
    x = self.net(obs.float())
    mu = self.linear_mu(x)
    # std = self.linear_std(x).clamp(0.01, 5.0)

    log_std = self.linear_log_std(x)
    log_std = log_std.clamp(-20, 2)
    std = log_std.exp()

    dist = Normal(mu, std)
    action = dist.rsample()
    log_prob = dist.log_prob(action)
    log_prob = log_prob.sum(dim=-1, keepdim=True)
    log_prob -= (2* (np.log(2) - action - F.softplus(-2*action))).sum(dim=-1, keepdim=True)

    action = torch.tanh(action) * self.max
    return action, log_prob

#### Soft actor-critic algorithm

In [17]:
class SACHER(LightningModule):

  def __init__(self, env_name, capacity=100_000, batch_size=256, lr=1e-3, 
               hidden_size=256, gamma=0.99, loss_fn=F.smooth_l1_loss, optim=AdamW, 
               samples_per_epoch=1_000, tau=0.05, alpha=0.02, her=0.8):

    super().__init__()
    self.automatic_optimization = False  # Disable automatic optimization
    self.env = create_environment(env_name)

    ag_size = self.env.observation_space['achieved_goal'].shape[0]
    dg_size = self.env.observation_space['desired_goal'].shape[0]
    obs_size = self.env.observation_space['observation'].shape[0]

    action_dims = self.env.action_space.shape[0]
    max_action = self.env.action_space.high

    self.q_net1 = DQN(hidden_size, obs_size + dg_size, action_dims)
    self.q_net2 = DQN(hidden_size, obs_size + dg_size, action_dims)
    self.policy = GradientPolicy(hidden_size, obs_size + dg_size, action_dims, max_action)

    self.target_policy = copy.deepcopy(self.policy)
    self.target_q_net1 = copy.deepcopy(self.q_net1)
    self.target_q_net2 = copy.deepcopy(self.q_net2)

    self.buffer = ReplayBuffer(capacity=capacity, her_probability=her)

    self.save_hyperparameters()

    while len(self.buffer) < self.hparams.samples_per_epoch * 2:
      print(f"{len(self.buffer)} samples in experience buffer. Filling...")
      self.play_episodes()

  @torch.no_grad()
  def play_episodes(self, policy=None):
      state, _ = self.env.reset()
      done = False
      truncate = False
      while not (done or truncate):
        desired_state = np.hstack([state['observation'], state['desired_goal']])
        achieved_state = np.hstack([state['observation'], state['achieved_goal']])
        
        if policy and random.random() > 0.1:
          action, _ = self.policy(desired_state)
          action = action.cpu().numpy()
        else:
          action = self.env.action_space.sample()
          
        next_state, reward, done, truncate, info = self.env.step(action)

        next_desired_state = np.hstack([next_state['observation'], next_state['desired_goal']])
        next_achieved_state = np.hstack([next_state['observation'], next_state['achieved_goal']])

        # Desired goal.
        exp = (desired_state, action, reward, done, next_desired_state)
        self.buffer.append(exp)

        # Achieved goal.
        reward = self.env.compute_reward(next_state['achieved_goal'], next_state['achieved_goal'], info)
        exp = (achieved_state, action, reward, done, next_achieved_state)
        self.buffer.append(exp, her=True)

        state = next_state

  def forward(self, x):
    output = self.policy(x)
    return output

  def configure_optimizers(self):
    # q_net_parameters = itertools.chain(self.q_net1.parameters(), self.q_net2.parameters())
    # q_net_optimizer = self.hparams.optim(q_net_parameters, lr=self.hparams.lr)
    # policy_optimizer = self.hparams.optim(self.policy.parameters(), lr=self.hparams.lr)
    # return [q_net_optimizer, policy_optimizer]
    q_net1_optimizer = self.hparams.optim(self.q_net1.parameters(), lr=self.hparams.lr)
    q_net2_optimizer = self.hparams.optim(self.q_net2.parameters(), lr=self.hparams.lr)
    policy_optimizer = self.hparams.optim(self.policy.parameters(), lr=self.hparams.lr)
    return [q_net1_optimizer, q_net2_optimizer, policy_optimizer]

  def train_dataloader(self):
    dataset = RLDataset(self.buffer, self.hparams.samples_per_epoch)
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=self.hparams.batch_size,
    )
    return dataloader

  def training_step(self, batch, batch_idx):
    states, actions, rewards, dones, next_states = batch
    rewards = rewards.unsqueeze(1)
    dones = dones.unsqueeze(1)
    opt_q_net1, opt_q_net2, opt_policy = self.optimizers()

    # if optimizer_idx == 0:

    action_values1 = self.q_net1(states, actions)
    action_values2 = self.q_net2(states, actions)

    target_actions, target_log_probs = self.target_policy(next_states)

    next_action_values = torch.min(
        self.target_q_net1(next_states, target_actions),
        self.target_q_net2(next_states, target_actions)
    )
    next_action_values[dones] = 0.0

    expected_action_values = rewards + self.hparams.gamma * (next_action_values - self.hparams.alpha * target_log_probs)

    q_loss1 = self.hparams.loss_fn(action_values1, expected_action_values)
    q_loss2 = self.hparams.loss_fn(action_values2, expected_action_values)
    opt_q_net1.zero_grad()
    q_loss1.backward(retain_graph=True)
    opt_q_net1.step()
    opt_q_net2.zero_grad()
    q_loss2.backward()
    opt_q_net2.step()
    q_loss_total = q_loss1 + q_loss2
    self.log("episode/Q-Loss", q_loss_total)
    
    # q_loss_total = q_loss1 + q_loss2
    # self.log("episode/Q-Loss", q_loss_total)
      # return q_loss_total

    # elif optimizer_idx == 1:

    actions, log_probs = self.policy(states)

    action_values = torch.min(
        self.q_net1(states, actions),
        self.q_net2(states, actions)
    )

    policy_loss = (self.hparams.alpha * log_probs - action_values).mean()
    opt_policy.zero_grad()
    policy_loss.backward()
    opt_policy.step()
    self.log("episode/Policy Loss", policy_loss)
      # return policy_loss

  def on_train_epoch_end(self):
    self.play_episodes(policy=self.policy)

    polyak_average(self.q_net1, self.target_q_net1, tau=self.hparams.tau)
    polyak_average(self.q_net2, self.target_q_net2, tau=self.hparams.tau)
    polyak_average(self.policy, self.target_policy, tau=self.hparams.tau)

    self.log("episode/episode_return", self.env.return_queue[-1])

In [15]:
# Start tensorboard.
!rm -r /content/lightning_logs/
!rm -r /content/videos/
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs/

rm: cannot remove '/content/lightning_logs/': No such file or directory
rm: cannot remove '/content/videos/': No such file or directory


ERROR: Failed to start `tensorboard`: [Errno 8] Exec format error:
'tensorboard'

In [18]:
algo = SACHER('FetchReach-v1', lr=1e-3, alpha=0.2, tau=0.1)

trainer = Trainer(
    # gpus=num_gpus, 
    max_epochs=4_000,
    log_every_n_steps=1
)

trainer.fit(algo)

/home/akhters/.local/lib/python3.8/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment FetchReach-v1 is out of date. You should consider upgrading to version `v2`.
  logger.deprecation(
/home/akhters/.local/lib/python3.8/site-packages/gymnasium_robotics/envs/robot_env.py:361: UserWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.warn(
/home/akhters/.local/lib/python3.8/site-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4 folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/home/akhters/.loc

0 samples in experience buffer. Filling...
Moviepy - Building video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-0.mp4.
Moviepy - Writing video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-0.mp4
100 samples in experience buffer. Filling...
200 samples in experience buffer. Filling...
300 samples in experience buffer. Filling...
400 samples in experience buffer. Filling...
500 samples in experience buffer. Filling...
600 samples in experience buffer. Filling...
700 samples in experience buffer. Filling...
800 samples in experience buffer. Filling...
900 samples in experience buffer. Filling...
1000 samples in experience buffer. Filling...
1100 samples in experience buffer. Filling...
1200 samples in experience buffer. Filling...
1300 samples in experience buffer. Filling...
1400 samples in experience buffer. Filling...
1500 samples in experience buffer. Filling...
1600 samples in experience buffer. Filling...
1700 samples in experience buffer. Filling...
1800 samples in experience buffer. Filling...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type           | Params | Mode 
---------------------------------------------------------
0 | q_net1        | DQN            | 70.7 K | train
1 | q_net2        | DQN            | 70.7 K | train
2 | policy        | GradientPolicy | 71.4 K | train
3 | target_policy | GradientPolicy | 71.4 K | train
4 | target_q_net1 | DQN            | 70.7 K | train
5 | target_q_net2 | DQN            | 70.7 K | train
---------------------------------------------------------
425 K     Trainable params
0         Non-trainable params
425 K     Total params
1.702     Total estimated model params size (MB)


1900 samples in experience buffer. Filling...


...

Moviepy - Done !
Moviepy - video ready /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3850.mp4
Moviepy - Building video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3900.mp4.
Moviepy - Writing video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3900.mp4



Moviepy - Done !
Moviepy - video ready /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3900.mp4
Moviepy - Building video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3950.mp4.
Moviepy - Writing video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3950.mp4



Moviepy - Done !
Moviepy - video ready /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-3950.mp4
Moviepy - Building video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-4000.mp4.
Moviepy - Writing video /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-4000.mp4



Moviepy - Done !
Moviepy - video ready /home/akhters/Reinforcement Learning/advanced_rl_dqn_to_sac_complete-main/videos4/rl-video-episode-4000.mp4


`Trainer.fit` stopped: `max_epochs=4000` reached.
